In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from evidently.report import Report
from evidently.metric_preset import ClassificationPreset
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [4]:
df = pd.read_csv("datasets/student-dataset-v1.csv")

In [5]:
le = LabelEncoder()

df['Result'] = le.fit_transform(df['Result'] )

In [6]:
X = df.drop(columns=['Result'])  
y = df['Result']  

In [7]:
# Encode categorical variables 
X = pd.get_dummies(X)

In [8]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Logistic Regression model
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

LogisticRegression()

In [10]:
X_train['prediction'] = lr_model.predict_proba(X_train)[:, 1]
X_test['prediction'] = lr_model.predict_proba(X_test)[:, 1]

In [11]:
X_train['target'] = y_train
X_test['target'] = y_test

In [12]:
lr_class_report= Report(metrics=[ClassificationPreset()])
lr_class_report.run(reference_data=X_train, current_data=X_test)
# lr_class_report.show(mode='inline')

In [14]:
lr_class_report.save("json_reports/lr_report_v1.json")

In [16]:
# Load JSON data
with open('json_reports/lr_report_v1.json', 'r') as file:
    data = json.load(file)

In [18]:
#accessing json report data (getting accuracy)

lr_accuracy_v1 = None
metric_results = data['suite']['metric_results']

for result in metric_results:
    if 'current' in result:
        lr_accuracy_v1 = result['current'].get('accuracy')
        if lr_accuracy_v1 is not None:
            break

if lr_accuracy_v1 is not None:
    print("Logistic Regression Accuracy:", lr_accuracy_v1)
else:
    print("Accuracy not found in the JSON data.")

Logistic Regression v1 Accuracy: 0.7241379310344828
